In [1]:
from airport_names import get_all_routes
import datetime
import threading
import queue
routes = queue.Queue()
dates = queue.Queue()


def add_zero(number):
    if number < 10:
        number = '0' + str(number)
    else:
        number = str(number)
        
    return number


        


In [2]:
from py2neo import authenticate, Graph, Relationship
graph = Graph("http://localhost:7474/db/data/")
print ("connected")
from py2neo import Node, Relationship

connected


In [3]:
api = 'ja373863706848344628128972526671'
from skyscanner.skyscanner import FlightsCache
flights_cache_service = FlightsCache(api)

def get_prices_for_route_and_month(flights_cache_service,outbound, destination, partial_date):
    ret = flights_cache_service.get_grid_prices_by_date(
            market='IE',
            currency='EUR',
            locale='en-IE',
            originplace="-".join([outbound, 'SKY']),
            destinationplace="-".join([destination, 'SKY']),
            outbounddate=partial_date).parsed
    
    prices = (item['MinPrice'] if item else None for item in ret['Dates'][1])
    return prices

In [4]:
import threading
import flights



import pickledb
db = pickledb.load('flights.db', False)

def fill_routes(routes, dates):
    date = dates.get()
    
    for route in get_all_routes():
        routes.put( (date, route) )

In [ ]:

routes.qsize()

In [ ]:
5

In [5]:
def get_prices_worker(flights_cache_service, routes, pickleDB):
    
    while True:
                
        if routes.qsize() < 5000:
            if dates.empty():
                for i in range(1,7):
                    dates.put( str( datetime.date.today() + datetime.timedelta(i*365/12) )[:7] )
                    
           
            
            
            t = threading.Thread(target=fill_routes, args=(routes, dates))
            t.start()
            #print("Route Size:", routes.qsize())
            
            
        for _ in range(500):
            t = threading.Thread(target=worker, args=(flights_cache_service, routes, pickleDB))
            t.start()
           
        print("Route Size:", routes.qsize())
        break
        thread.sleep(60)
        
# args=(graph, out, dest, short_date,index, price)
def get_flight_node(graph, outbound, destination, partial_date,day, price):
    """
        Called by worker
        Needs 1) route 2) date 3) price
        If price update or create flight
        If not price delete
    """
    
    if day < 10:
        day = "0"+ str(day)
    else:
        day = str(day)
        
    flight_date = "".join([partial_date,  day])
    flight_name = "".join([outbound,destination,flight_date])
    flight = graph.find_one("Flight", "flight", flight_name)
    
    if price:
    
        if flight:
            flight['price'] = price
            flight.push()
        else:
            flight_node = Node("Flight", flight=flight_name, price=price)
            graph.create(flight)
            
    else:
        
        graph.delete(flight)
            
    
        

def worker(flights_cache_service, routes, pickleDB):
    partial, route = routes.get()
    out, dest = route
    
    try:
        grid = get_prices_for_route_and_month(flights_cache_service, out, dest, partial)
        short_date=partial[2:4]+partial[-2:]
        key = short_date+out+dest
        print("Key", key)
        pickleDB.set(key, grid)
        
        for index, price in enumerate(grid, start=1):
            set_prices_thread = threading.Thread(target=get_flight_node, args=(graph, out, dest, short_date,index, price))
            set_prices_thread.start()
    except Exception as e:
        print(e)
        
        with open('errors.txt', 'a+') as f:
            f.write("-".join(route))
            f.write("\n")

In [6]:
import datetime



dt_started = datetime.datetime.utcnow()

t = threading.Thread(target=get_prices_worker, args=(flights_cache_service, routes,db))
    #threads.append(t)
t.start()



Key 1702VIEHRK
Key 1702VIEPOZ
Key 1702VIEEMA
Key 1702VIEPUF
Key 1702VIECGN
Key 1702VIEUFA
Key 1702VIECRL
Key 1702VIEABZ
Key 1702VIESOU
Key 1702VIELBA
Key 1702VIELIS
Key 1702VIEDCM
Key 1702VIEDRS


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:41: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead


Key 1702VIEGLA
Key 1702VIEPMI
Key 1702VIEMPL
Key 1702VIEBIQ
Key 1702VIEGOA
Key 1702VIEMAD
Key 1702VIELTN
Key 1702VIEZAG
Key 1702VIEOSL
Key 1702VIESKG
Key 1702VIEAER
Key 1702VIELED
Key 1702VIEMAN
Key 1702VIEMLA
Key 1702VIEEIN
Key 1702VIECDG
Key 1702VIEKEF
Key 1702VIEPMO
Key 1702VIESVO
Key 1702VIEMMX


Exception in thread Thread-127:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\database\__init__.py", line 1125, in delete
    subgraph.__db_delete__(self)
AttributeError: 'NoneType' object has no attribute '__db_delete__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-8850b01a5720>", line 54, in get_flight_node
    graph.delete(flight)
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\database\__init__.py", line 443, in delete
    self.begin(autocommit=True).delete(subgraph)
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\database\__init__.py", line 1127, in delete
    raise TypeError("No method defined to delete object %r

Key 1702VIEBFS


...

Exception in thread Thread-537:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\packages\neo4j\v1\session.py", line 124, in session
    session = self.session_pool.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-8850b01a5720>", line 41, in get_flight_node
    flight = graph.find_one("Flight", "flight", flight_name)
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\util.py", line 82, in f_
    return f(*args, **kwargs)
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\database\__init__.py", line 513, in find_one
    return node_selection.first()
  File "C:\Program Files\Anaconda3\lib\sit

Key 1702VIEBLL
Key 1702VIEBHD


Exception in thread Thread-545:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\packages\neo4j\v1\session.py", line 124, in session
    session = self.session_pool.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-8850b01a5720>", line 41, in get_flight_node
    flight = graph.find_one("Flight", "flight", flight_name)
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\util.py", line 82, in f_
    return f(*args, **kwargs)
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\database\__init__.py", line 513, in find_one
    return node_selection.first()
  File "C:\Program Files\Anaconda3\lib\sit

Key 1702VIELDE


Exception in thread Thread-607:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\packages\neo4j\v1\session.py", line 124, in session
    session = self.session_pool.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-8850b01a5720>", line 41, in get_flight_node
    flight = graph.find_one("Flight", "flight", flight_name)
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\util.py", line 82, in f_
    return f(*args, **kwargs)
  File "C:\Program Files\Anaconda3\lib\site-packages\py2neo\database\__init__.py", line 513, in find_one
    return node_selection.first()
  File "C:\Program Files\Anaconda3\lib\sit

In [ ]:

today = datetime.datetime.now()
year, month = today.year, today.month
print(year, month)

In [ ]:
import datetime
print (datetime.date.today() + datetime.timedelta(6*365/12))#.isoformat()

In [ ]:
print("Route Size:", routes.qsize())

db.getall()

In [ ]:
db.get('2017-01PMINAP')